In [1]:
import os

os.environ['OMP_NUM_THREADS'] = '2'
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(42)


def generate_train_and_test(split=0.8, shuffle=True):
    iris = datasets.load_iris()
    data = iris.data[:100, :].astype(np.float32)
    data = preprocessing.minmax_scale(data) * 2 - 1
    label = np.zeros(100).astype(int)
    label[50:] = 1
    return train_test_split(data, label, train_size=split, shuffle=shuffle)


train_x, test_x, train_y, test_y = generate_train_and_test(split=0.2)
print('train sample and feature shape: ', train_x.shape)
print('first train sample: ', train_x[0, :], ' with label: ', train_y[0])

train sample and feature shape:  (20, 4)
first train sample:  [ 0.33333302 -0.24999988  0.804878    0.5294118 ]  with label:  1


In [2]:
from mindquantum import H, RZ, RX, X, Circuit, RY, UN


def encoder(n):
    c = Circuit()
    #create your new encoder here
    for i in range(n):
        c += RZ(f'x{i}').on(i)
    return c


enc = encoder(4).no_grad()
print("encoder circuit:")
print(enc)
print("\n")

def ansatz(n_qubits):
    c = Circuit()
    # create your new ansatz here:
    for i in range(n_qubits):
        c += RX(f"gamma{i}").on(i)
    return c


ans = ansatz(4)
print("ansatz circuit:")
print(ans)
if len(ans) > 15:
    raise ValueError(
        "Too many gates in the ansatz circuit, we limited it to 15 gate.")
tot = enc + ans
print("\ntotal circuit:")
print(tot)

encoder circuit:
q0: ──RZ(x0)──
              
q1: ──RZ(x1)──
              
q2: ──RZ(x2)──
              
q3: ──RZ(x3)──


ansatz circuit:
q0: ──RX(gamma0)──
                  
q1: ──RX(gamma1)──
                  
q2: ──RX(gamma2)──
                  
q3: ──RX(gamma3)──

total circuit:
q0: ──RZ(x0)────RX(gamma0)──
                            
q1: ──RZ(x1)────RX(gamma1)──
                            
q2: ──RZ(x2)────RX(gamma2)──
                            
q3: ──RZ(x3)────RX(gamma3)──


/home/xuxs/gitee/mindquantum/mindquantum/__init__.py:49: UserWarning: [NOTE] Current simulator thread is 2. If your simulation is slow, set OMP_NUM_THREADS to a appropriate number according to your model.
  warnings.warn(


In [3]:
from mindquantum.ops import QubitOperator
from mindquantum import Hamiltonian

hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [2, 3]]
print(hams)

[1.0 [Z2] , 1.0 [Z3] ]


In [4]:
from mindquantum import Simulator
sim = Simulator('projectq', tot.n_qubits)
f_g_ops = sim.get_expectation_with_grad(hams, tot,encoder_params_name=enc.params_name,ansatz_params_name=ans.params_name, parallel_worker=4)

In [5]:
p0 = np.random.uniform(size=len(ans.params_name))
f_g_ops(train_x[:1], p0)

(array([[0.70372061+0.j, 0.82195572+0.j]]),
 array([[[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]]),
 array([[[ 1.23778604e-18+0.j, -1.04856580e-17+0.j, -7.10476815e-01+0.j,
           0.00000000e+00+0.j],
         [-4.34601462e-18+0.j,  1.72178009e-17+0.j,  6.22260857e-17+0.j,
          -5.69551395e-01+0.j]]]))

In [6]:
import mindspore as ms
from mindquantum.nn import MQLayer
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(42)
net = MQLayer(f_g_ops)

In [7]:
from mindspore import Tensor
print('QNN output: \n', net(Tensor(train_x[:4])))
print('\nweight: \n', net.weight.asnumpy())

QNN output: 
 [[0.99998873 0.9999043 ]
 [0.99998873 0.9999043 ]
 [0.99998873 0.9999043 ]
 [0.99998873 0.9999043 ]]

weight: 
 [ 0.00495268 -0.00754737  0.00474223 -0.01383794]


In [8]:
import mindspore.dataset as ds
ms.set_seed(42)

batch = 5
train_loader = ds.NumpySlicesDataset({
    'feats': train_x,
    'labs': train_y
},
                                     shuffle=False).batch(batch)
test_loader = ds.NumpySlicesDataset({
    'feats': test_x,
    'labs': test_y
}).batch(batch)

In [9]:
loss = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
opti = ms.nn.Adam(net.trainable_params(), learning_rate=1e-1)
monitor = ms.train.callback.LossMonitor(16)

In [10]:
model = ms.Model(net, loss, opti, metrics={'Acc': ms.nn.Accuracy()})
model.train(50, train_loader, callbacks=[monitor], dataset_sink_mode=False)

[WARNING] ME(119501:140177980307264,MainProcess):2021-09-07-16:05:11.507.493 [mindspore/train/model.py:194] In acc mode, the eval_network must be defined.


epoch: 4 step: 4, loss is 0.68296826
epoch: 8 step: 4, loss is 0.60619485
epoch: 12 step: 4, loss is 0.58345073
epoch: 16 step: 4, loss is 0.59326154
epoch: 20 step: 4, loss is 0.5955518
epoch: 24 step: 4, loss is 0.593799
epoch: 28 step: 4, loss is 0.59321004
epoch: 32 step: 4, loss is 0.59344286
epoch: 36 step: 4, loss is 0.59354985
epoch: 40 step: 4, loss is 0.5935019
epoch: 44 step: 4, loss is 0.5934632
epoch: 48 step: 4, loss is 0.59345645


In [11]:
predict = np.argmax(ms.ops.Softmax()(model.predict(ms.Tensor(test_x))), axis=1)
corr = model.eval(test_loader, dataset_sink_mode=False)
print(corr)

{'Acc': 0.475}
